# Tu mejor portatil

## Competición Kaggle

Nuestro jefe estaba buscando un@s maquinas para obtener datos de la competencia y poder aplicarle los conocimientos obtenidos para asignar precios a nuestra tienda de "Cositas-Markt". Lamentablemente se habían ido de vacaciones y nos lo ha pedido a nosotr@s … 💥🪓🔪

Nos toca arremangarnos y aplicar los conocimientos obtenidos en ML para obtener un modelo de predicción de precios de portátiles en función de sus marcas y prestaciones para poder lanzarlos a un precio competitivo al mercado.

Para ello contamos con un dataset con los siguientes campos:

1 Company- String -Laptop Manufacturer 2 Product -String -Brand and Model 3 TypeName -String -Type (Notebook, Ultrabook, Gaming, etc.) 4 Inches -Numeric- Screen Size 5 ScreenResolution -String- Screen Resolution 6 Cpu- String -Central Processing Unit (CPU) 7 Ram -String- Laptop RAM 8 Memory -String- Hard Disk / SSD Memory 9 GPU -String- Graphics Processing Units (GPU) 10 OpSys -String- Operating System 11 Weight -String- Laptop Weight 12 Price_euros -Numeric- Price (Euro)

### Importaciones

In [129]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error, mean_squared_error


## Funciones


In [130]:
def extraer_ssd_gb(memory):
    if not isinstance(memory, str):
        return 0  # si hay un valor nulo
    
    # dividir las que tienen +
    parts = memory.split("+")
    
    total_ssd = 0
    for part in parts:
        part = part.strip()  # quitar espacios al principio/final
        
        # solo nos interesa si tiene "SSD"
        if "SSD" in part:
            # quitar "SSD" y espacios
            num_unit = part.replace("SSD", "").strip()
            
            # detectar unidad
            if "TB" in num_unit:
                num = float(num_unit.replace("TB","").strip())
                total_ssd += num * 1024  # convertir TB a GB
            else:  # asumimos GB
                num = float(num_unit.replace("GB","").strip())
                total_ssd += num
    return total_ssd

def extraer_hdd_gb(memory):
    
    if not isinstance(memory, str):
        return 0
    
    parts = memory.split("+")
    total_hdd = 0
    
    for part in parts:
        part = part.strip()
        
        if "HDD" in part:
            num_unit = part.replace("HDD", "").strip()
            
            if "TB" in num_unit:
                num = float(num_unit.replace("TB","").strip())
                total_hdd += num * 1024
            else:
                num = float(num_unit.replace("GB","").strip())
                total_hdd += num
    return total_hdd

def extraer_hybrid_gb(memory):
    if not isinstance(memory, str):
        return 0
    
    parts = memory.split("+")
    total_hybrid = 0
    
    for part in parts:
        part = part.strip()
        
        if "Hybrid" in part:
            num_unit = part.replace("Hybrid","").strip()
            
            if "TB" in num_unit:
                num = float(num_unit.replace("TB","").strip())
                total_hybrid += num * 1024
            else:
                num = float(num_unit.replace("GB","").strip())
                total_hybrid += num
                
    return total_hybrid

def extraer_flash_gb(memory):
    if not isinstance(memory, str):
        return 0
    
    parts = memory.split("+")
    total_flash = 0
    
    for part in parts:
        part = part.strip()
        
        if "Flash Storage" in part:
            num_unit = part.replace("Flash Storage","").strip()
            
            if "TB" in num_unit:
                num = float(num_unit.replace("TB","").strip())
                total_flash += num * 1024
            else:
                num = float(num_unit.replace("GB","").strip())
                total_flash += num
                
    return total_flash

def cpu_marca(cpu_str):
    if "Intel" in cpu_str:
        return "Intel"
    elif "AMD" in cpu_str:
        return "AMD"
    else:
        return "Other"

def cpu_serie(cpu_str):
    lista_series = ["i3", "i5", "i7", "i9", "M", "Atom", "Celeron", "Pentium", "Xeon",
                   "A6", "A8", "A9", "A10", "FX", "Ryzen", "E-Series"]
    for serie in lista_series:
        if serie in cpu_str:
            return serie
    return "Other"

def cpu_ghz(cpu_str):
    try:
        for part in cpu_str.split(): #parte el string
            if "GHz" in part: #coge la parte que tiene GHz
                return float(part.replace("GHz", "")) #le quita el GHz y lo convierte en float
        return 0.0 #si no tiene GHz da 0.0
    except:
        return 0.0 #si da error da 0.0
    
def gpu_marca(gpu_str):
    if "Intel" in gpu_str:
        return "Intel"
    elif "Nvidia" in gpu_str:
        return "Nvidia"
    elif "AMD" in gpu_str:
        return "AMD"
    else:
        return "Other"

def gpu_tipo(gpu_str):
    gpu = gpu_str.strip()  # elimina espacios al inicio/final
    if "Intel" in gpu:
        if "HD Graphics" in gpu:
            return "HD Graphics"
        elif "UHD Graphics" in gpu:
            return "UHD Graphics"
        elif "Iris Plus" in gpu:
            return "Iris Plus"
        elif "Iris" in gpu:
            return "Iris"
        else:
            return "Other Intel"
    elif "Nvidia" in gpu:
        if "GeForce" in gpu:
            return "GeForce"
        elif "Quadro" in gpu:
            return "Quadro"
        else:
            return "Other Nvidia"
    elif "AMD" in gpu:
        if "Radeon" in gpu:
            return "Radeon"
        elif "RX" in gpu:
            return "RX"
        elif "FirePro" in gpu:
            return "FirePro"
        else:
            return "Other AMD"
    else:
        return "Other"
    



### Lectura de CSV y primeros pasos

In [131]:
df = pd.read_csv("data/train.csv")
df

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,89,973,Acer,Chromebook C738T-C2EJ,2 in 1 Convertible,11.6,IPS Panel Touchscreen 1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,32GB Flash Storage,Intel HD Graphics 400,Chrome OS,1.25kg,389.00
908,716,1018,Toshiba,Portege Z30-C-16H,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.2kg,1195.00
909,517,1071,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,8GB,1TB HDD,AMD Radeon R5,Windows 10,2.4kg,449.00
910,891,965,Dell,Latitude 7480,Ultrabook,14.0,Full HD 1920x1080,Intel Core i7 7600U 2.8GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.36kg,1775.00


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [133]:
df.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros'],
      dtype='object')

De todas estas columnas, hay que saber cuales son importantes para el precio de un laptop. Investigando un poco:

1. CPU
2. GPU
3. RAM
4. SSD_GB
5. PPI (Esto es resolucion por pulgadas)
6. Memoria / HDD_GB / Hybrid_GB / Flash_GB
7. Notebook, gaming...
8. SO
9. Peso
10. Marca

De primeras: columnas como RAM y Weight deberian ser Int y Float y en Memory hay que mirar si estan los valores en TB o GB porque 1TB es mas que 256 GB y podria dar a error. 

In [134]:
df["Ram"].unique()

array(['8GB', '4GB', '16GB', '12GB', '6GB', '32GB', '2GB', '24GB'],
      dtype=object)

In [135]:
df["Ram"] = df["Ram"].str.replace("GB", "").astype(float)


In [136]:
df

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8.0,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4.0,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8.0,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16.0,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8.0,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,89,973,Acer,Chromebook C738T-C2EJ,2 in 1 Convertible,11.6,IPS Panel Touchscreen 1366x768,Intel Celeron Dual Core N3060 1.6GHz,4.0,32GB Flash Storage,Intel HD Graphics 400,Chrome OS,1.25kg,389.00
908,716,1018,Toshiba,Portege Z30-C-16H,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4.0,128GB SSD,Intel HD Graphics 520,Windows 10,1.2kg,1195.00
909,517,1071,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,8.0,1TB HDD,AMD Radeon R5,Windows 10,2.4kg,449.00
910,891,965,Dell,Latitude 7480,Ultrabook,14.0,Full HD 1920x1080,Intel Core i7 7600U 2.8GHz,8.0,256GB SSD,Intel HD Graphics 620,Windows 10,1.36kg,1775.00


In [137]:
df["Weight"].unique()

array(['2.36kg', '2kg', '1.2kg', '4.42kg', '1.26kg', '2.26kg', '2.4kg',
       '1.3kg', '2.5kg', '2.65kg', '2.02kg', '2.2kg', '1.95kg', '1.75kg',
       '1.88kg', '1.74kg', '2.23kg', '3kg', '0.69kg', '2.71kg', '3.35kg',
       '1.78kg', '2.13kg', '2.32kg', '2.18kg', '2.20kg', '2.1kg', '1.7kg',
       '2.3kg', '1.91kg', '2.31kg', '1.4kg', '1.48kg', '2.04kg', '1.54kg',
       '1.86kg', '1.98kg', '1.47kg', '2.67kg', '1.85kg', '1.5kg',
       '0.97kg', '1.6kg', '1.68kg', '2.09kg', '2.33kg', '3.49kg',
       '3.78kg', '1.36kg', '2.77kg', '1.29kg', '1.62kg', '1.05kg',
       '2.05kg', '1.9kg', '1.28kg', '1.49kg', '2.62kg', '2.0kg', '1.63kg',
       '1.37kg', '2.73kg', '2.69kg', '1.25kg', '4.14kg', '2.9kg',
       '1.23kg', '2.37kg', '1.10kg', '1.90kg', '2.63kg', '1.39kg',
       '2.8kg', '2.94kg', '1.43kg', '1.22kg', '2.21kg', '1.32kg', '2.6kg',
       '1.45kg', '1.14kg', '4.36kg', '1.8kg', '1.1kg', '3.3kg', '3.2kg',
       '4.2kg', '1.35kg', '2.19kg', '1.96kg', '4.0kg', '1.84kg', '1.65kg',


In [138]:
df["Weight"] = df["Weight"].str.replace("kg", "").astype(float)

In [139]:
df

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8.0,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4.0,1TB HDD,Intel HD Graphics 505,Windows 10,2.00,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8.0,256GB SSD,Intel HD Graphics 520,Windows 7,1.20,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16.0,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8.0,256GB SSD,Intel HD Graphics 520,Windows 7,1.26,1713.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,89,973,Acer,Chromebook C738T-C2EJ,2 in 1 Convertible,11.6,IPS Panel Touchscreen 1366x768,Intel Celeron Dual Core N3060 1.6GHz,4.0,32GB Flash Storage,Intel HD Graphics 400,Chrome OS,1.25,389.00
908,716,1018,Toshiba,Portege Z30-C-16H,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4.0,128GB SSD,Intel HD Graphics 520,Windows 10,1.20,1195.00
909,517,1071,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,8.0,1TB HDD,AMD Radeon R5,Windows 10,2.40,449.00
910,891,965,Dell,Latitude 7480,Ultrabook,14.0,Full HD 1920x1080,Intel Core i7 7600U 2.8GHz,8.0,256GB SSD,Intel HD Graphics 620,Windows 10,1.36,1775.00


Con memoria creo una columnna para el tipo de memoria y otra para la cantidad

In [140]:
df["Memory"].unique()

array(['1TB HDD', '256GB SSD', '256GB SSD +  1TB HDD',
       '512GB SSD +  2TB HDD', '128GB SSD +  1TB HDD', '128GB SSD',
       '1TB SSD', '64GB Flash Storage', '2TB HDD', '500GB HDD',
       '1.0TB Hybrid', '16GB Flash Storage', '256GB Flash Storage',
       '180GB SSD', '32GB Flash Storage', '512GB SSD',
       '512GB SSD +  1TB HDD', '256GB SSD +  500GB HDD',
       '256GB SSD +  2TB HDD', '1TB SSD +  1TB HDD', '64GB SSD',
       '128GB Flash Storage', '256GB SSD +  1.0TB Hybrid', '32GB SSD',
       '512GB Flash Storage', '240GB SSD', '256GB SSD +  256GB SSD',
       '16GB SSD', '8GB SSD', '64GB Flash Storage +  1TB HDD',
       '1.0TB HDD', '1TB HDD +  1TB HDD', '512GB SSD +  512GB SSD',
       '128GB SSD +  2TB HDD', '32GB HDD', '512GB SSD +  1.0TB Hybrid'],
      dtype=object)

hay diferentes tipos y algunos estan combinados. Hay que separar por SSD, HDD, Flash, Hybrid y ademas ver las que tienen 2 tipos. Ojo tb con el 1.0. Ademas paso 1TB a 1024 GB y asi tengo todo en la misma unidad

In [141]:
df["SSD_GB"] = df["Memory"].apply(extraer_ssd_gb)

In [142]:
df["HDD_GB"] = df["Memory"].apply(extraer_hdd_gb)

In [143]:
df["Hybrid_GB"] = df["Memory"].apply(extraer_hybrid_gb)

In [144]:
df["Flash_GB"] = df["Memory"].apply(extraer_flash_gb)

In [145]:
df[["Memory","SSD_GB","HDD_GB","Hybrid_GB","Flash_GB"]].sample(20)


,Memory,SSD_GB,HDD_GB,Hybrid_GB,Flash_GB
187,256GB SSD,256.0,0.0,0.0,0.0
634,256GB SSD,256.0,0.0,0.0,0.0
589,128GB SSD + 1TB HDD,128.0,1024.0,0.0,0.0
607,1TB HDD,0.0,1024.0,0.0,0.0
567,256GB SSD,256.0,0.0,0.0,0.0
641,256GB SSD,256.0,0.0,0.0,0.0
451,256GB SSD,256.0,0.0,0.0,0.0
806,256GB SSD,256.0,0.0,0.0,0.0
256,128GB SSD + 1TB HDD,128.0,1024.0,0.0,0.0
714,1TB HDD,0.0,1024.0,0.0,0.0


In [146]:
df["Hybrid_GB"].unique()

array([   0., 1024.])

In [147]:
df["Flash_GB"].unique()

array([  0.,  64.,  16., 256.,  32., 128., 512.])

In [148]:
df.head(20)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,SSD_GB,HDD_GB,Hybrid_GB,Flash_GB
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8.0,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36,749.00,0.0,1024.0,0.0,0.0
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4.0,1TB HDD,Intel HD Graphics 505,Windows 10,2.00,449.00,0.0,1024.0,0.0,0.0
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8.0,256GB SSD,Intel HD Graphics 520,Windows 7,1.20,1460.00,256.0,0.0,0.0,0.0
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16.0,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42,2868.99,256.0,1024.0,0.0,0.0
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8.0,256GB SSD,Intel HD Graphics 520,Windows 7,1.26,1713.37,256.0,0.0,0.0,0.0
5,867,376,Asus,ZenBook Flip,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12.0,512GB SSD + 2TB HDD,Nvidia GeForce GT 940MX,Windows 10,2.26,1099.00,512.0,2048.0,0.0,0.0
6,1136,60,Asus,X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux),Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4.0,256GB SSD,Intel HD Graphics 520,Linux,2.00,415.00,256.0,0.0,0.0,0.0
7,609,1157,MSI,GP62M 7RDX,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8.0,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.40,1142.80,128.0,1024.0,0.0,0.0
8,474,110,Asus,UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8.0,256GB SSD,Nvidia GeForce 940MX,Windows 10,1.30,1193.00,256.0,0.0,0.0,0.0
9,838,1263,Asus,Rog GL553VE-FY052T,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16.0,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1050 Ti,Windows 10,2.50,1600.00,256.0,1024.0,0.0,0.0


Aparte de las memorias, otros puntos importantes para el precio de un laptop seran la pantalla, la grafica y la cpu

### Pantalla. 
Limpiar Screen Resolution para ver que tipos hay y que resoluciones

In [149]:
df["ScreenResolution"].unique()

array(['1366x768', 'Full HD 1920x1080', 'IPS Panel 4K Ultra HD 3840x2160',
       'Full HD / Touchscreen 1920x1080', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'IPS Panel Touchscreen 1920x1200',
       '4K Ultra HD / Touchscreen 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Retina Display 2880x1800', 'IPS Panel 1366x768',
       'IPS Panel Quad HD+ / Touchscreen 3200x1800',
       'Quad HD+ / Touchscreen 3200x1800', '4K Ultra HD 3840x2160',
       'IPS Panel 2560x1440', '1600x900', 'Touchscreen 2560x1440',
       'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel Retina Display 2304x1440', '1440x900',
       'IPS Panel Retina Display 2560x1600', 'Touchscreen 2256x1504',
       '2560x1440', 'IPS Panel Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2560x1440', '1920x1080',
       'Touchscreen 2400x1600', 'IPS Panel Full HD 2160x1440',
       'Quad HD+ 3200x1800',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x21

La resolucion va al final de cada linea. 

In [150]:
df['ScreenResolution_clean'] = df['ScreenResolution'].apply(lambda x: x.split()[-1])


In [151]:
df[['Width_px', 'Height_px']] = df['ScreenResolution_clean'].str.split('x', expand=True)


In [152]:
df['Width_px'] = df['Width_px'].astype(int)
df['Height_px'] = df['Height_px'].astype(int)


con Width_px, Height_px y Inches calculo el PPI 

In [153]:
df['PPI'] = ((df['Width_px']**2 + df['Height_px']**2)**0.5) / df['Inches']

Ahora, tipo de pantalla IPS, Touchscreen. Y despues el full HD de la misma manera

In [154]:
df['IPS'] = df['ScreenResolution'].str.contains('IPS', case=False).astype(int)
df['Touchscreen'] = df['ScreenResolution'].str.contains('Touchscreen', case=False).astype(int)
df['FullHD'] = df['ScreenResolution'].str.contains('Full HD', case=False).astype(int)
df['QuadHD'] = df['ScreenResolution'].str.contains('Quad HD', case=False).astype(int)
df['UltraHD_4K'] = df['ScreenResolution'].str.contains('4K|Ultra HD', case=False).astype(int)
df['Retina'] = df['ScreenResolution'].str.contains('Retina', case=False).astype(int)



In [155]:
df.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,...,ScreenResolution_clean,Width_px,Height_px,PPI,IPS,Touchscreen,FullHD,QuadHD,UltraHD_4K,Retina
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8.0,1TB HDD,...,1366x768,1366,768,100.454670,0,0,0,0,0,0
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4.0,1TB HDD,...,1366x768,1366,768,100.454670,0,0,0,0,0,0
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8.0,256GB SSD,...,1920x1080,1920,1080,165.632118,0,0,1,0,0,0
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16.0,256GB SSD + 1TB HDD,...,3840x2160,3840,2160,282.423996,1,0,0,0,1,0
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8.0,256GB SSD,...,1920x1080,1920,1080,176.232574,0,1,1,0,0,0


### CPU

Limpiar el string CPU. Sacar fabricante, tipo de procesador y frecuencia

In [156]:
df["Cpu"].unique()

array(['Intel Core i7 7500U 2.7GHz',
       'Intel Pentium Quad Core N4200 1.1GHz',
       'Intel Core i5 6200U 2.3GHz', 'Intel Core i7 7700HQ 2.8GHz',
       'Intel Core i5 6300U 2.4GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Pentium Quad Core N3710 1.6GHz',
       'Intel Core i7 8550U 1.8GHz', 'AMD A9-Series A9-9420 3GHz',
       'Intel Pentium Dual Core 4405U 2.1GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 7100U 2.4GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Xeon E3-1535M v6 3.1GHz',
       'Intel Atom x5-Z8550 1.44GHz', 'AMD A10-Series 9620P 2.5GHz',
       'Intel Core i5 7300U 2.6GHz',
       'Intel Celeron Dual Core N3350 1.1GHz',
       'Intel Core i7 6500U 2.5GHz', 'Intel Core i3 6006U 2.0GHz',
       'Intel Celeron Dual Core 3205U 1.5GHz',
       'AMD A12-Series 9720P 3.6GHz',
       'Intel Celeron Dual Core N3050 1.6GHz',
       'Intel Core i3 7130U 2.7GHz', 'Intel Core i5 7300HQ 2.5GHz',
       'Intel Core i5 7200U 2.7GHz', 'Intel Core i7 7820HQ 2.9GHz

Hay 2 marcas. Intel y AMD. Tipos de procesador hay bastantes. La frecuencia siempre va al final y acaba en GHz. 

Marcas

In [157]:
df["CPU_Marca"] = df["Cpu"].apply(cpu_marca)

In [158]:
df["CPU_Marca"].count()

np.int64(912)

In [159]:
df["CPU_Serie"] = df["Cpu"].apply(cpu_serie)

In [160]:
df["CPU_Serie"].count()

np.int64(912)

In [161]:
df["CPU_GHz"] = df["Cpu"].apply(cpu_ghz)

In [162]:
df["CPU_GHz"].count()

np.int64(912)

### GPU



In [163]:
df["Gpu"].unique()

array(['AMD Radeon R7 M445', 'Intel HD Graphics 505',
       'Intel HD Graphics 520', 'Nvidia GeForce GTX 1070',
       'Nvidia GeForce GT 940MX', 'Nvidia GeForce GTX 1050',
       'Nvidia GeForce 940MX', 'Nvidia GeForce GTX 1050 Ti',
       'Intel HD Graphics 405', 'Intel UHD Graphics 620',
       'AMD Radeon 530', 'Intel HD Graphics 510', 'Intel HD Graphics 620',
       'Nvidia Quadro M2200M', 'Nvidia GeForce MX150',
       'Intel HD Graphics 400', 'Nvidia Quadro M1200',
       'Nvidia GeForce 920MX', 'Intel HD Graphics 500', 'AMD Radeon 520',
       'AMD Radeon R5 M330', 'AMD Radeon R5 M420X', 'Intel HD Graphics',
       'Intel HD Graphics 630', 'AMD Radeon R5 M430',
       'Intel Iris Pro Graphics', 'Nvidia GeForce GTX 940MX',
       'Intel HD Graphics 515', 'Nvidia GeForce GTX 1080',
       'AMD Radeon R2 Graphics', 'Nvidia GeForce 150MX', 'AMD Radeon R5',
       'Nvidia GeForce GTX 1050Ti', 'AMD Radeon R9 M385',
       'Nvidia GeForce 930MX', 'Nvidia GeForce GTX 1060',
       'Nv

Mismo caso que con las CPU. Hay qye sacar la marca y el modelo

In [164]:
df["GPU_Marca"] = df["Gpu"].apply(gpu_marca)

In [165]:
df["GPU_Marca"].count()

np.int64(912)

In [166]:
df["GPU_Tipo"] = df["Gpu"].apply(gpu_tipo)

In [167]:
df["GPU_Tipo"].count()

np.int64(912)

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      912 non-null    int64  
 1   laptop_ID               912 non-null    int64  
 2   Company                 912 non-null    object 
 3   Product                 912 non-null    object 
 4   TypeName                912 non-null    object 
 5   Inches                  912 non-null    float64
 6   ScreenResolution        912 non-null    object 
 7   Cpu                     912 non-null    object 
 8   Ram                     912 non-null    float64
 9   Memory                  912 non-null    object 
 10  Gpu                     912 non-null    object 
 11  OpSys                   912 non-null    object 
 12  Weight                  912 non-null    float64
 13  Price_euros             912 non-null    float64
 14  SSD_GB                  912 non-null    fl

In [169]:
df.to_csv("data/train_trabajado.csv", index= False )